In [ ]:
import tensorflow as tf
import numpy as np

import matplotlib.pylab as plt
%matplotlib inline

### Hello World

In [ ]:
a = tf.add(4,2)

with tf.Session() as sess:
    print(sess.run(a))

### Computation graph

In [ ]:
tf.reset_default_graph()

x = 4
y = 2

op1 = tf.add(x, y)
op2 = tf.multiply(x, y)
op3 = tf.pow(op2, op1)

with tf.Session() as sess:
    print(sess.run(op3))

try:
    from utils import show_current_graph
    show_current_graph()
except:
    pass

### Tensors and dimensions

In [ ]:
from itertools import combinations

data = [2, [1, 2], [1, 2, 3]]

def get_result(x, y):
    op1 = tf.add(x, y)
    op2 = tf.add(x, y)
    op3 = tf.add(op2, op1)

    with tf.Session() as sess:
        return sess.run(op3)
    
for x, y in combinations(data, 2):
    try:
        result = get_result(x,y)
        print('OK', x, 'and', y, '->', result)
    except:
        print('Fail', x, y)

### Variables

In [ ]:
a = tf.Variable([2, 3], name='vector')
b = tf.Variable([[0, 1], [2, 3]], name='matrix')
W = tf.Variable(tf.zeros(shape=[784, 10]), name='tensor')

assign_op = a.assign([1, 2])

init_ab = tf.variables_initializer([a, b])
init_all_vars = tf.global_variables_initializer() # also you gan get all defined vars using tf.global_variables() func

with tf.Session() as sess:
    sess.run(a.initializer) # Initialize a single variable
    print(a.eval())
    # print(b.eval()) # If you try to execute this op you will get a exception
    
    sess.run(init_ab) # Initialize only a subset (a,b) of virables
    print(b.eval())
    
    sess.run(init_all_vars) # Initialize all variables at once
    print(W.eval())
    
    sess.run(assign_op)
    print(a.eval())

### Session

In [ ]:
W = tf.Variable(10)

sess1 = tf.Session()
sess2 = tf.Session()

sess1.run(W.initializer)
sess2.run(W.initializer)

print(sess1.run(W.assign_add(10)))
print(sess2.run(W.assign_add(100)))

sess1.close()
sess2.close()

In [ ]:
# Interactive session
sess = tf.InteractiveSession()

a = tf.Variable(2)
b = tf.Variable(2)
c = a + b

tf.global_variables_initializer().run() # Init all variables

print(c.eval())

sess.close()

### Placeholders

In [ ]:
a = tf.placeholder(tf.float32, shape=[3])
b = tf.constant([5, 5, 5], tf.float32)
c = a + b

with tf.Session() as sess:
    # print(sess.run(c)) # Error because we a doesn't have any value
    
    feed_dict = {a: [5, 2, 3]}
    print(sess.run(c, feed_dict=feed_dict))

### Simple regression mode

In [ ]:
tf.reset_default_graph()

N_EPOCHS = 20

from sklearn.datasets import make_regression

# Step 1: init dataset
X_data, y_data = make_regression(n_features=1, noise=10, random_state=42)
n_samples = X_data.shape[0]

# Step 2: create placeholders for X and Y
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')

# Step 3: create weight and bias, initialized to 0
w = tf.get_variable('weights', initializer=tf.constant(0.0))
b = tf.get_variable('bias', initializer=tf.constant(0.0))

# Step 4: build model to predict Y
Y_predicted = w * X + b 

# Step 5: use the squared error as the loss function
# you can use either mean squared error or Huber loss
loss = tf.square(Y - Y_predicted, name='loss')
# loss = utils.huber_loss(Y, Y_predicted)

# Step 6: using gradient descent with learning rate of 0.001 to minimize loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)


with tf.Session() as sess:
    # Step 7: initialize the necessary variables, in this case, w and b
    sess.run(tf.global_variables_initializer()) 
    
    # Step 8: train the model
    for i in range(1, N_EPOCHS+1): 
        total_loss = 0
        for x, y in zip(X_data, y_data):
            # Session execute optimizer and fetch values of loss
            _, l = sess.run([optimizer, loss], feed_dict={X: x, Y:y}) 
            total_loss += l
        print('Epoch {0}: {1}'.format(i, total_loss/n_samples))
        
        if i % 5 == 0:
            # Step 9: output the values of w and b
            w_out, b_out = sess.run([w, b])

            plt.close()
            plt.plot(X_data, y_data, 'bo', label='Real data')
            plt.plot(X_data, X_data * w_out + b_out, 'r', label='Predicted data')
            plt.legend()
            plt.show()

### TF MNIST

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

tf.logging.set_verbosity(tf.logging.ERROR)
tf.reset_default_graph()

# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 30

# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets('data/mnist', one_hot=True)
X_batch, Y_batch = mnist.train.next_batch(batch_size)

In [ ]:
plt.imshow(X_batch[0].reshape((28, 28)), 'gray')

In [ ]:
# Step 2: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 
# each lable is one hot vector.
X = tf.placeholder(tf.float32, [batch_size, 784], name='image') 
Y = tf.placeholder(tf.int32, [batch_size, 10], name='label')

# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
w = tf.get_variable(name='weights', shape=(784, 10), initializer=tf.random_normal_initializer())
b = tf.get_variable(name='bias', shape=(1, 10), initializer=tf.zeros_initializer())

# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
logits = tf.matmul(X, w) + b 

# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
loss = tf.reduce_mean(entropy) # computes the mean over all the examples in the batch
# loss = tf.reduce_mean(-tf.reduce_sum(tf.nn.softmax(logits) * tf.log(Y), reduction_indices=[1]))

# Step 6: define training op
# using gradient descent with learning rate of 0.01 to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    n_batches = int(mnist.train.num_examples/batch_size)
    
    # train the model n_epochs times
    for i in range(n_epochs): 
        total_loss = 0

        for j in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch = sess.run([optimizer, loss], {X: X_batch, Y:Y_batch}) 
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    # test the model
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0

    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run(accuracy, {X: X_batch, Y:Y_batch})
        total_correct_preds += accuracy_batch    

    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

In [ ]:
try:
    from utils import show_current_graph
    show_current_graph()
except:
    pass

### Keras MNIST

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(10, activation='softmax', input_shape=(784,)))
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model.fit(mnist.train.images, mnist.train.labels,
          batch_size=batch_size, epochs=5,
          validation_data = (mnist.test.images, mnist.test.labels))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_test = mnist.test.labels.argmax(axis=1)
y_predicted_scores = model.predict(mnist.test.images)
y_predicted = y_predicted_scores.argmax(axis=1)

print('Classification report\n')
print(classification_report(y_test, y_predicted))

In [ ]:
from utils import print_confusion_matrix
print_confusion_matrix(confusion_matrix(y_test, y_predicted), range(0, 10))
plt.show()

In [ ]:
# show incorrect predictions

from itertools import islice

incorrect_classified = y_test != y_predicted

incorrect_classified_images = mnist.test.images[incorrect_classified]
incorrect_classified_gt = y_test[incorrect_classified]
incorrect_classified_scores = y_predicted_scores[incorrect_classified]

top_n_examples = 5
iter_incorrect = islice(zip(incorrect_classified_images,
                            incorrect_classified_gt,
                            incorrect_classified_scores),
                        top_n_examples)

for im, label, score in iter_incorrect:
    print('Correct label:', label)
    predicted_label = score.argmax()
    print('Predicted:', predicted_label, 'with score', score[predicted_label])
    plt.imshow(im.reshape((28, 28)), 'gray')
    plt.show()

# More examples

[DeepDream](http://nbviewer.jupyter.org/github/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb)

[Neural networks](https://github.com/aymericdamien/TensorFlow-Examples)

[TF official examples](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/learn)